# TASK 1

In [1]:
class HashTable:
    def __init__(self, size=10):
        self.size = size
        self.table = [None] * size

    def hash_function(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        index = self.hash_function(key)
        if self.table[index] is None:
            self.table[index] = []
        self.table[index].append((key, value))

    def search(self, key):
        index = self.hash_function(key)
        if self.table[index] is not None:
            for kvp in self.table[index]:
                if kvp[0] == key:
                    return kvp[1]
        return None

    def delete(self, key):
        index = self.hash_function(key)
        if self.table[index] is not None:
            for i, kvp in enumerate(self.table[index]):
                if kvp[0] == key:
                    del self.table[index][i]
                    return True
        return False

# Example usage
ht = HashTable()
ht.insert("key1", "value1")
ht.insert("key2", "value2")
print(ht.search("key1"))  # Outputs: value1
ht.delete("key1")
print(ht.search("key1"))  # Outputs: None


value1
None


# TASK 2

In [2]:
def binary_search(arr, x):
    low = 0
    high = len(arr) - 1
    iterations = 0
    upper_bound = None

    while low <= high:
        mid = (low + high) // 2
        iterations += 1
        if arr[mid] == x:
            return (iterations, arr[mid])
        elif arr[mid] < x:
            low = mid + 1
        else:
            high = mid - 1
            upper_bound = arr[mid]

    if upper_bound is None and low < len(arr):
        upper_bound = arr[low]
    return (iterations, upper_bound)

# Example usage
sorted_array = [1.1, 2.2, 3.3, 4.4, 5.5]
value_to_find = 3.0
print(binary_search(sorted_array, value_to_find))  # Outputs: (2, 3.3)


(3, 3.3)


# TASK 3

Step 1: Implement the substring search algorithms

Boyer-Moore Algorithm

In [3]:
def boyer_moore_search(text, pattern):
    def bad_character_table(pattern):
        table = {}
        length = len(pattern)
        for i in range(length - 1):
            table[pattern[i]] = length - i - 1
        return table
    
    bad_char_table = bad_character_table(pattern)
    m, n = len(pattern), len(text)
    s = 0  # shift of the pattern with respect to text
    iterations = 0
    
    while s <= n - m:
        j = m - 1
        while j >= 0 and pattern[j] == text[s + j]:
            j -= 1
        if j < 0:
            return (iterations, s)  # found pattern at index s
        else:
            s += bad_char_table.get(text[s + j], m)
        iterations += 1
    
    return (iterations, -1)  # pattern not found


Knuth-Morris-Pratt (KMP) Algorithm

In [4]:
def kmp_search(text, pattern):
    def compute_lps(pattern):
        lps = [0] * len(pattern)
        length = 0
        i = 1
        while i < len(pattern):
            if pattern[i] == pattern[length]:
                length += 1
                lps[i] = length
                i += 1
            else:
                if length != 0:
                    length = lps[length - 1]
                else:
                    lps[i] = 0
                    i += 1
        return lps
    
    lps = compute_lps(pattern)
    i = j = 0
    iterations = 0
    
    while i < len(text):
        if pattern[j] == text[i]:
            i += 1
            j += 1
        
        if j == len(pattern):
            return (iterations, i - j)  # found pattern at index i-j
            j = lps[j - 1]
        elif i < len(text) and pattern[j] != text[i]:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1
        iterations += 1
    
    return (iterations, -1)  # pattern not found


Rabin-Karp Algorithm

In [5]:
def rabin_karp_search(text, pattern, q=101):
    d = 256
    m = len(pattern)
    n = len(text)
    p = 0  # hash value for pattern
    t = 0  # hash value for text
    h = 1
    iterations = 0

    for i in range(m - 1):
        h = (h * d) % q

    for i in range(m):
        p = (d * p + ord(pattern[i])) % q
        t = (d * t + ord(text[i])) % q

    for i in range(n - m + 1):
        if p == t:
            if text[i:i+m] == pattern:
                return (iterations, i)  # found pattern at index i

        if i < n - m:
            t = (d * (t - ord(text[i]) * h) + ord(text[i + m])) % q
            if t < 0:
                t = t + q
        iterations += 1
    
    return (iterations, -1)  # pattern not found


Step 2: Read the text files and define test cases

In [6]:
def read_file_with_encodings(file_path, encodings=['utf-8-sig', 'latin1', 'cp1252']):
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                return file.read()
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"Unable to read the file {file_path} with the given encodings.")

# Read the text files
text1 = read_file_with_encodings('/Users/michalpokracki/Downloads/article1.txt')
text2 = read_file_with_encodings('/Users/michalpokracki/Downloads/article2.txt')

# Define substrings for testing
existing_substring1 = "existing substring in article 1"
non_existing_substring1 = "this substring does not exist in article 1"

existing_substring2 = "existing substring in article 2"
non_existing_substring2 = "this substring does not exist in article 2"


Step 3: Measure the execution time using timeit

In [7]:
import timeit

def measure_time(search_function, text, pattern):
    wrapped = lambda: search_function(text, pattern)
    time = timeit.timeit(wrapped, number=1000)  # measure time over 1000 executions
    return time

# Measure times for article 1
time_bm_exist1 = measure_time(boyer_moore_search, text1, existing_substring1)
time_bm_non_exist1 = measure_time(boyer_moore_search, text1, non_existing_substring1)

time_kmp_exist1 = measure_time(kmp_search, text1, existing_substring1)
time_kmp_non_exist1 = measure_time(kmp_search, text1, non_existing_substring1)

time_rk_exist1 = measure_time(rabin_karp_search, text1, existing_substring1)
time_rk_non_exist1 = measure_time(rabin_karp_search, text1, non_existing_substring1)

# Measure times for article 2
time_bm_exist2 = measure_time(boyer_moore_search, text2, existing_substring2)
time_bm_non_exist2 = measure_time(boyer_moore_search, text2, non_existing_substring2)

time_kmp_exist2 = measure_time(kmp_search, text2, existing_substring2)
time_kmp_non_exist2 = measure_time(kmp_search, text2, non_existing_substring2)

time_rk_exist2 = measure_time(rabin_karp_search, text2, existing_substring2)
time_rk_non_exist2 = measure_time(rabin_karp_search, text2, non_existing_substring2)


Step 4: Compare the results

In [8]:
results_markdown = f"""
# Substring Search Algorithm Efficiency

## Article 1

| Algorithm       | Existing Substring Time (ms) | Non-Existing Substring Time (ms) |
|-----------------|------------------------------|----------------------------------|
| Boyer-Moore     | {time_bm_exist1:.6f}              | {time_bm_non_exist1:.6f}            |
| Knuth-Morris-Pratt | {time_kmp_exist1:.6f}          | {time_kmp_non_exist1:.6f}           |
| Rabin-Karp      | {time_rk_exist1:.6f}              | {time_rk_non_exist1:.6f}            |

## Article 2

| Algorithm       | Existing Substring Time (ms) | Non-Existing Substring Time (ms) |
|-----------------|------------------------------|----------------------------------|
| Boyer-Moore     | {time_bm_exist2:.6f}              | {time_bm_non_exist2:.6f}            |
| Knuth-Morris-Pratt | {time_kmp_exist2:.6f}          | {time_kmp_non_exist2:.6f}           |
| Rabin-Karp      | {time_rk_exist2:.6f}              | {time_rk_non_exist2:.6f}            |

## Conclusion

Based on the above results, the fastest algorithm for each article and in general is determined.
- For Article 1, the fastest algorithm is: {'Boyer-Moore' if min(time_bm_exist1, time_kmp_exist1, time_rk_exist1) == time_bm_exist1 else 'Knuth-Morris-Pratt' if min(time_bm_exist1, time_kmp_exist1, time_rk_exist1) == time_kmp_exist1 else 'Rabin-Karp'}
- For Article 2, the fastest algorithm is: {'Boyer-Moore' if min(time_bm_exist2, time_kmp_exist2, time_rk_exist2) == time_bm_exist2 else 'Knuth-Morris-Pratt' if min(time_bm_exist2, time_kmp_exist2, time_rk_exist2) == time_kmp_exist2 else 'Rabin-Karp'}
- Overall, the fastest algorithm is: {'Boyer-Moore' if min(time_bm_exist1 + time_bm_exist2, time_kmp_exist1 + time_kmp_exist2, time_rk_exist1 + time_rk_exist2) == time_bm_exist1 + time_bm_exist2 else 'Knuth-Morris-Pratt' if min(time_bm_exist1 + time_bm_exist2, time_kmp_exist1 + time_kmp_exist2, time_rk_exist1 + time_rk_exist2) == time_kmp_exist1 + time_kmp_exist2 else 'Rabin-Karp'}
"""

# Save the results to a markdown file
with open('results.md', 'w') as file:
    file.write(results_markdown)

print("Results have been saved to 'results.md'")


Results have been saved to 'results.md'
